In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign

In [5]:
# date0='0709_1157'
# date1='0709_1557'
# date2='0709_1934'
# dates=[date0,date1,date2]
# plate = 13
date0='0701_1957'
date1='0701_2357'
date2='0702_0357'
date3='0702_0757'
date4='0702_1157'
date5='0702_1557'
date6='0702_1957'
date7='0702_2357'
plate=13
dates=[date0,date1,date2,date3,date4]

In [6]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(get_path(date,plate,True,extension='_full_labeled.csv'),
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [25]:
tabs_labeled=[]
for date in dates:
    tabs_labeled.append(pd.read_csv(f'Data/graph_{date}_{plate}_full_labeled.csv',
                                    converters={'origin_pos' : transform_list,'end_pos' : transform_list,'pixel_list' : ast.literal_eval}))

In [7]:
nx_graphs=[]
poss=[]
for tab in tabs_labeled:
    nx_graph,pos=generate_nx_graph(tab,labeled=True)
    nx_graphs.append(nx_graph)
    poss.append(pos)

In [8]:
nx_graph_clean=[]
for graph in nx_graphs:
    S = [graph.subgraph(c).copy() for c in nx.connected_components(graph)]
    len_connected=[len(nx_graph.nodes) for nx_graph in S]
    nx_graph_clean.append(S[np.argmax(len_connected)])

In [9]:
skeletons=[]
for nx_graph in nx_graph_clean:
    skeletons.append(generate_skeleton(nx_graph,dim=(20687, 45554)))

In [10]:
factor = 5
final_pictures = [compress_skeleton(skeletons[i],factor) for i in range(len(skeletons))]

In [11]:
connections_growth_pattern = [whole_movement_identification(nx_graph_clean[i],nx_graph_clean[i+1],poss[i],poss[i+1]) for i in range(len(dates)-1)]

0.0
0.14084507042253522
0.28169014084507044
0.4225352112676056
0.5633802816901409
0.704225352112676
0.8450704225352113
0.9859154929577465
0.0
0.13262599469496023
0.26525198938992045
0.3978779840848806
0.5305039787798409
0.6631299734748011
0.7957559681697612
0.9283819628647215
0.0
0.12224938875305623
0.24449877750611246
0.36674816625916873
0.4889975550122249
0.6112469437652812
0.7334963325183375
0.8557457212713936
0.9779951100244498
0.0
0.11467889908256881
0.22935779816513763
0.3440366972477064
0.45871559633027525
0.573394495412844
0.6880733944954128
0.8027522935779816
0.9174311926605505


In [12]:
connections = [c[0] for c in connections_growth_pattern]
growth_pattern = [c[1] for c in connections_growth_pattern]

In [ ]:
def pinpoint_anastomosis(nx_graph_tm1,nx_grapht,equ):
    anastomosis=[]
    origins=[]
    tips = [node for node in nx_graph_tm1.nodes if nx_graph_tm1.degree(node)==1]
    for node in nx_grapht.nodes:
        if nx_grapht.degree(node)>=3 and node in equ.keys() and equ[node] in tips:
            anastomosis.append(node)
            origins.append(equ[node])
    return(anastomosis,origins)

In [13]:
growing_tips=[[node for node in growth_pattern[i].keys() if len(growth_pattern[i][node])>=20] for i in range(len(growth_pattern))]

In [14]:
degree3_nodes = [[node for node in nx_graph.nodes if nx_graph.degree(node)>=3] for nx_graph in nx_graph_clean]

In [15]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

NameError: name 't' is not defined

In [39]:
anastomosiss=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i])[0] for i in range (len(dates)-1)]
origins=[pinpoint_anastomosis(nx_graph_clean[i],nx_graph_clean[i+1], connections[i])[1] for i in range (len(dates)-1)]

In [49]:
2295 in anastomosiss[t]

True

In [42]:
t=1
tp1=t+1

In [ ]:
plot_t_tp1(origins[t],anastomosiss[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
plot_t_tp1(growing_tips[t],growing_tips[t],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
t=3
tp1=t+1
plot_t_tp1([2180],[2180],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5,)

In [ ]:
plot_t_tp1(degree3_nodes[t],degree3_nodes[tp1],poss[t],poss[tp1],final_pictures[t],final_pictures[tp1],compress=5)

In [19]:
def make_growth_picture_per_tip(pixels_from_tip,pos,shape=(20700,45600),factor=10,max_growth=200,min_growth=10,per_tip=True):
    final_picture = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    number_tips = np.zeros(shape=(shape[0]//factor,shape[1]//factor))
    for tip in pixels_from_tip.keys():
        growth=len(pixels_from_tip[tip])
        x=min(round(pos[tip][0]/factor),shape[0]//factor-1)
        y=min(round(pos[tip][1]/factor),shape[1]//factor-1)
        if growth<=max_growth:
#             print(number_tips)
            if growth>=min_growth:
                number_tips[x,y]+=1
                final_picture[x,y]+=growth
#         print(growth,beginx,endx)    
#     for x in range(shape[0]//factor):
#         if x%1==0:
#             print(x/2070)
#         for y in range(shape[1]//factor):
#             beginx = x*factor
#             endx=(x+1)*factor
#             beginy = y*factor
#             endy=(y+1)*factor
#             tips_in_frame = [tip for tip in pixels_from_tip.keys() if (beginx<pos[tip][0]<endx) and (beginy<pos[tip][1]<endy)]
#             #shouls be improved, len is not a good indicator of actual length...
#             growth_in_frame = [len(pixels_from_tip[tip]) for tip in tips_in_frame]
#             final_picture[x,y]=np.mean(growth_in_frame)
    if per_tip:
        return(final_picture/(number_tips+(number_tips==0).astype(np.int)),number_tips)
    else:
        return(final_picture,number_tips)

In [35]:
final_pictures_growth = [np.log(make_growth_picture_per_tip(growth_pattern[i],poss[i],factor=500,max_growth=4000,per_tip=True,min_growth=0)[0]+1) for i in range (len(growth_pattern))]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(final_pictures_growth[0])

In [27]:
final_pictures[2].shape

(41, 91)

In [32]:
import cv2
import numpy as np
 

frameSize_origin = (41,91)

out = cv2.VideoWriter('Data\output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 6, frameSize)

for i in range(0,255):
    img = np.ones((frameSize[0],frameSize[1], 3), dtype=np.uint8)*i
    print(img)
    out.write(img)

out.release()

In [ ]:
import cv2
import numpy as np
 

frameSize_origin = (41,91)
real_size=(91,91)
out = cv2.VideoWriter('Data\output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 6, real_size)

for i in range(0,len(final_pictures)):
    img = final_pictures[i]
    img = cv2.resize(img,real_size)
    out.write(img)

out.release()

In [ ]:
import cv2
import numpy as np
 

frameSize = final_pictures[0].shape

out = cv2.VideoWriter('Data\output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1, frameSize)

for i in range(len(final_pictures)):
    img = final_pictures[i]
    out.write(img)

out.release()

In [58]:
pos_problem=[11241, 20621]
xbegin=pos_problem[0]-1000
ybegin=pos_problem[1]-1000
xend=pos_problem[0]+1000
yend=pos_problem[1]+1000
skeletons_small=[]
for skeleton in skeletons:
    skeletons_small.append(skeleton[xbegin:xend,ybegin:yend])
node_smalls=[]
for i,nx_graph in enumerate(nx_graph_clean):
    node_smalls.append([node for node in nx_graph.nodes if (xbegin<poss[i][node][0]<xend and ybegin<poss[i][node][1]<yend and nx_graph.degree(node)>=1)])

In [96]:
kernel = np.ones((5,5),np.uint8)
cv2.dilate(width2.astype(np.uint8),kernel,iterations = 2)

In [101]:
skeletons_small_dilated=[cv2.dilate(skeleton.todense().astype(np.uint8),kernel,iterations = 1) for skeleton in skeletons_small]

In [ ]:
t=1
tp1=t+1

In [ ]:
plot_t_tp1(node_smalls[tp1],node_smalls[tp1],poss[tp1],poss[tp1],skeletons_small_dilated[tp1],skeletons_small_dilated[tp1],shift=(xbegin,ybegin))

In [72]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1,relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save=''):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    for node in node_list_t:
        t = ax.text((pos_t[node][1]-shift[1])//compress, (pos_t[node][0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=5,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            t = ax.text((pos_tp1[node][1]-shift[1])//compress, (pos_tp1[node][0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=5,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
    else:
        plt.show()